In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers,Sequential  
import matplotlib.pyplot as plt
import numpy as np
image_dim  = 256
batch_dim= 32

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "tea sickness dataset",
    shuffle =True,
    seed = 123,
    image_size = ( image_dim, image_dim ),
    batch_size = batch_dim
    
)

class_name = dataset.class_names

batch_count = len( dataset)
def train_test_split(dataset , train_s = 0.8 , validation_s =0.1, test_s = 0.1, shuffle = True , Shuffle_size = 10000, seed = 123 ):
    assert( train_s + validation_s + test_s) == 1
    if shuffle:
     dataset = dataset.shuffle( Shuffle_size , seed )   
     train_length = int(train_s* batch_count)
     validation_length = int(validation_s* batch_count)
     test_length = int(test_s*batch_count)
     
     train_ds = dataset.take(train_length)
     remaining_ds = dataset.skip(train_length)
     validation_ds = remaining_ds.take(validation_length)
     test_ds = remaining_ds.skip(validation_length)
     
     return train_ds, validation_ds, test_ds
 
 
 
train_ds , validation_ds , test_ds = train_test_split( dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

resize_image=tf.keras.Sequential ([
 layers.Resizing(image_dim,image_dim ),
    layers.Rescaling(1.0/255),
       
    
])
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.5),
    layers.RandomContrast(0.2),
])

train_d = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

colour = 3
input_shape = (batch_count, image_dim, image_dim, colour)
model = models.Sequential([
    
    resize_image, 
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    
    layers.Dense(8, activation='softmax'),

])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.build(input_shape=(batch_count, image_dim, image_dim, colour))
model.summary()

Found 885 files belonging to 8 classes.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)       │ (28, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (28, 254, 254, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (28, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (28, 125, 125, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (28, 62, 62, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (28, 60, 60, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (28, 30, 30, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (28, 28, 28, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (28, 14, 14, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (28, 12, 12, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (28, 6, 6, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (28, 4, 4, 128)        │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (28, 2, 2, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (28, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (28, 256)              │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (28, 8)                │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 420,392 (1.60 MB)

 Trainable params: 420,392 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
training_history = model.fit(
    train_ds,
    epochs=20,
    batch_size=batch_count,
    verbose=1,
    validation_data=validation_ds,
    
)

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 320ms/step - accuracy: 0.1685 - loss: 2.0560 - val_accuracy: 0.3019 - val_loss: 1.6377
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 6s 290ms/step - accuracy: 0.2560 - loss: 1.5717 - val_accuracy: 0.3208 - val_loss: 1.4681
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 339ms/step - accuracy: 0.3387 - loss: 1.4759 - val_accuracy: 0.3019 - val_loss: 1.5566
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 347ms/step - accuracy: 0.4031 - loss: 1.3837 - val_accuracy: 0.4717 - val_loss: 1.1861
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 335ms/step - accuracy: 0.4538 - loss: 1.2859 - val_accuracy: 0.4906 - val_loss: 1.1246
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 331ms/step - accuracy: 0.5212 - loss: 1.1224 - val_accuracy: 0.6226 - val_loss: 1.0938
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 328ms/step - accuracy: 0.6039 - loss: 1.0082 - val_accuracy: 0.6604 - val_loss: 0.8943
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 346ms/step - accuracy: 0.6649 - loss: 0.8712 - val_accuracy: 0

In [4]:
score = model.evaluate(test_ds) 

score 

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.9187 - loss: 0.3146


[0.47113949060440063, 0.890625]

In [5]:
from tensorflow.keras.models import load_model
model.save('C:/Users/Arnav Singh/OneDrive/Desktop/tea leaf//m1.keras')